In [2]:
# Install required libraries
# You can install these packages using pip if not installed:
# !pip install pandas sqlite3 pandasql

import sqlite3
import pandas as pd

# Set working directory
import os
os.chdir("/Users/jade/Desktop/Humana/Training")

# Load datasets using pandas
features_data = pd.read_csv("Afeatures.csv")
control_data = pd.read_csv("Controlpoint.csv")
cost_data = pd.read_csv("CostUt.csv")
demograph_data = pd.read_csv("Demographics.csv")
condition_data = pd.read_csv("mcondition.csv")
detail_data = pd.read_csv("mdetail.csv")
claims_data = pd.read_csv("mclaims.csv")
target_data = pd.read_csv("Tmembers.csv")
members_data = pd.read_csv("mdata.csv")
pharmacy_data = pd.read_csv("PU.csv")
quality_data = pd.read_csv("QD.csv")
sales_data = pd.read_csv("SC.csv")
social_data = pd.read_csv("Socialh.csv")
web_data = pd.read_csv("WA.csv")

/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_5732/3976071838.py:19: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_data = pd.read_csv("mclaims.csv")
/var/folders/wm/7c949vv90fl9xprhlvgygqw40000gn/T/ipykernel_5732/3976071838.py:23: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_data = pd.read_csv("QD.csv")


In [3]:
import re
def convert_tenure(value):
    # Check if value is a string before processing
    if isinstance(value, str):
        # If the format is 'X - Y YEARS', calculate the midpoint
        if '-' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 2:  # Ensure there are two numbers for the range
                return (float(numbers[0]) + float(numbers[1])) / 2
        # If the format is 'X+ YEARS', take the number before the '+'
        elif '+' in value and 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:  # Ensure there's at least one number
                return float(numbers[0])
        # If there's just 'X YEARS', return the number
        elif 'YEARS' in value:
            numbers = re.findall(r'\d*\.?\d+', value)
            if len(numbers) == 1:
                return float(numbers[0])
        # Default case for strings that don't match the expected format
        return None
    # If value is already a float or an int, return it as is
    elif isinstance(value, (float, int)):
        return value
    # Default return for unexpected types (e.g., NaN, None)
    return None

# Applying the function to the column
members_data['tenure_band'] = members_data['tenure_band'].apply(convert_tenure)
members_data['disabled_ind'] = members_data['disabled_ind'].replace({'Y': 1, 'N': 0})
members_data['dual_eligible_ind'] = members_data['dual_eligible_ind'].replace({'Y': 1, 'N': 0})
members_data['lis_ind']= members_data['lis_ind'].replace({'Y': 1, 'N': 0})

In [4]:
merged_data = pd.merge(members_data, pharmacy_data, on='id', how='inner')
merged_data = pd.merge(merged_data, social_data, on='id', how='inner')
merged_data = pd.merge(merged_data, web_data, on='id', how='inner')
demograph_data = pd.read_csv("Demographics.csv")

In [5]:
new_target=target_data.drop(['product_type','calendar_year','plan_category'],axis=1)
merged_data = pd.merge(merged_data, new_target, on='id', how='inner')
merged_data = pd.merge(merged_data, features_data, on='id', how='inner')
merged_data = pd.merge(merged_data, control_data, on='id', how='inner')
merged_data = pd.merge(merged_data, cost_data, on='id', how='inner')
demograph_data=demograph_data.drop(columns=['lang_spoken_cd', 'rucc_category'])
merged_data = pd.merge(merged_data, demograph_data, on='id', how='inner')


In [6]:
sales_data['channel'] = sales_data['channel'].map({
    'Field':0,
    'Consumer Direct': 1,
    'Partner Call Center': 2,
    'DMS Telesales': 3,
    'Brokerage': 4
})
merged_data = pd.merge(merged_data, sales_data, on='id', how='inner')
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 231 entries, consec_tenure_month to channel
dtypes: float64(224), int64(7)
memory usage: 2.6 GB


In [7]:
region_mapping = {region: idx for idx, region in enumerate(detail_data['region'].fillna('Unknown').unique())}
race_mapping = {race: idx for idx, race in enumerate(detail_data['race'].fillna('Unknown').unique())}

# Print the mappings to see what was assigned (optional)
print("Region Mapping:", region_mapping)
print("Race Mapping:", race_mapping)

# Map the 'region' column to numeric values and overwrite the original column
detail_data['region'] = detail_data['region'].fillna('Unknown').map(region_mapping)

# Map the 'race' column to numeric values and overwrite the original column
detail_data['race'] = detail_data['race'].fillna('Unknown').map(race_mapping)
detail_data=detail_data.drop(columns=['mco_contract_nbr','state_of_residence','county_of_residence'])
detail_data['generic_grouper'] = detail_data['generic_grouper'].fillna(10008)
detail_data['generic_grouper'] = detail_data['generic_grouper'].replace({'Y': 1, 'N': 0})
detail_data['unattributed_provider'] = detail_data['unattributed_provider'].fillna(10008)
detail_data['unattributed_provider'] = detail_data['unattributed_provider'].replace({'Y': 1, 'N': 0})
detail_data['sex_cd'] = detail_data['sex_cd'].fillna(10008)
detail_data['sex_cd'] = detail_data['sex_cd'].replace({'F': 1, 'M': 0,'U':10008})
detail_data['veteran_ind'] = detail_data['veteran_ind'].fillna(10008)
detail_data['veteran_ind'] = detail_data['veteran_ind'].replace({'Y': 1, 'N': 0})
merged_data = pd.merge(merged_data, detail_data, on='id', how='inner')

Region Mapping: {'Florida': 0, 'Gulf South': 1, 'South Central': 2, 'Southeast': 3, 'Midwest': 4, 'Northeast': 5, 'Intermountain': 6, 'Central': 7, 'Pacific Southwest': 8, 'Puerto Rico': 9, 'Unknown': 10}
Race Mapping: {'Unknown': 0, 'BLACK': 1, 'N AMERICAN NATIVE': 2, 'WHITE': 3, 'ASIAN': 4, 'HISPANIC': 5, 'UNKNOWN': 6, 'OTHER': 7}


In [8]:
condition_aggregated = condition_data.groupby('id').agg({
    'hcc_model_type': lambda x: ', '.join(x),  # Join diseases as a single string
}).reset_index()
condition_aggregated
def remove_duplicates(hcc_model_type):
    # Split the string into a list, remove duplicates using set, and join it back into a string
    unique_terms = ', '.join(sorted(set(hcc_model_type.split(', '))))
    return unique_terms

# Apply the function to remove duplicates from 'cond_desc' column
condition_aggregated['hcc_model_type'] = condition_aggregated['hcc_model_type'].apply(remove_duplicates)
merged_data = pd.merge(merged_data, condition_aggregated, on='id', how='outer')

In [9]:
merged_data['hcc_model_type'] = merged_data['hcc_model_type'].fillna(0)

# Step 2: Replace specific strings with numeric values
merged_data['hcc_model_type'] = merged_data['hcc_model_type'].replace({'U': 0, 'MEDICAL': 1, 'ESRD': 2})
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 241 entries, consec_tenure_month to hcc_model_type
dtypes: float64(224), int64(17)
memory usage: 2.7 GB


In [102]:
#claims_data = claims_data.drop(columns = ['dos_year','clm_unique_key','serv_date_skey'])
#claims_data = claims_data.groupby('id').apply(lambda group: group.notna().sum())

In [10]:
# Step 1: Replace 'Y' with 1 and everything else (including NaN) with 0
claims_data_numeric = claims_data.replace({'Y': 1}).fillna(0)

# Step 2: Group by 'id' and sum the numeric values to count the number of 'Y's in each column
claims_data_group = claims_data_numeric.groupby('id', as_index=False).sum()

In [12]:
claims_data_group=claims_data_group.drop(columns = ['dos_year','clm_unique_key','serv_date_skey'])

In [13]:
claims_data_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329101 entries, 0 to 1329100
Data columns (total 24 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   id                        1329101 non-null  int64  
 1   pcp_visit                 1329101 non-null  float64
 2   annual_wellness           1329101 non-null  float64
 3   humana_paf                1329101 non-null  float64
 4   preventative_visit        1329101 non-null  float64
 5   comp_physical_exam        1329101 non-null  float64
 6   ihwa                      1329101 non-null  float64
 7   fqhc_visit                1329101 non-null  float64
 8   telehealth                1329101 non-null  float64
 9   endocrinologist_visit     1329101 non-null  float64
 10  oncolologist_visit        1329101 non-null  float64
 11  radiologist_visit         1329101 non-null  float64
 12  podiatrist_visit          1329101 non-null  float64
 13  ophthalmologist_visit     1

In [14]:
merged_data = pd.merge(merged_data, claims_data_group, on='id', how='outer')

In [15]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1527904 entries, 0 to 1527903
Columns: 264 entries, consec_tenure_month to er_visit
dtypes: float64(247), int64(17)
memory usage: 3.0 GB


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Assuming merged_data is already available

# Step 1: Prepare the data
# Replace 'target_column' with the actual name of your target column
X = merged_data.drop(columns=['preventive_visit_gap_ind'])  # Features
y = merged_data['preventive_visit_gap_ind']  # Target

# Step 1.1: Impute missing values (NaN) with 0
X = X.fillna(0)
y = y.fillna(0)

# Step 2: Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train and evaluate models

# XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", xgb_acc)
print(classification_report(y_test, y_pred_xgb))

from sklearn.metrics import roc_auc_score

# Assuming y_test is the true labels and y_pred_xgb are the predicted labels
y_pred_xgb_proba = xgb_model.predict_proba(X_test)[:, 1]  # Get the predicted probabilities for the positive class

# Calculate AUC
xgb_auc = roc_auc_score(y_test, y_pred_xgb_proba)
print("XGBoost AUC:", xgb_auc)

/Users/jade/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:06:38] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.6943854493571263
              precision    recall  f1-score   support

           0       0.69      0.81      0.74    168453
           1       0.70      0.56      0.62    137128

    accuracy                           0.69    305581
   macro avg       0.70      0.68      0.68    305581
weighted avg       0.70      0.69      0.69    305581

XGBoost AUC: 0.7591357501006044


In [17]:
quality_data = quality_data.drop(columns = ['measurement_year','measure_name','measure_desc','base_event_date','eligible_cnt'])
quality_data['measure_type'] = quality_data['measure_type'].fillna(10008)
quality_data['measure_type'] = quality_data['measure_type'].replace({'Patient Safety': 1, 'Patient Experience': 0, 'HEDIS':2})
#quality_data = quality_data.groupby('id').apply(lambda group: group.notna().sum())

In [25]:
quality_data_group=quality_data.groupby('id').sum()

In [26]:
quality_data_group

,measure_type,compliant_cnt
id,,
1,9,30.32
2,6,6.00
3,36,39.00
4,24,14.00
6,2,1.00
...,...,...
1999995,0,24.00
1999996,20,6.00
1999997,23,38.64


In [28]:
merged_data = pd.merge(merged_data, quality_data_group, on='id', how='outer')

In [29]:
merged_data 

,consec_tenure_month,all_mm_tenure,tenure_band,dual_eligible_ind,disabled_ind,lis_ind,id,rx_overall_pmpm_cost,rx_overall_net_paid_pmpm_cost,rx_overall_coins_pmpm_cost,...,cardiologist_visit,gastroenterologist_visit,orthopedist_visit,obgyn_visit,nephroloogist_visit,pulmonologist_visit,urgent_care_visit,er_visit,measure_type,compliant_cnt
0,6,6,0.25,0,0,0,1551235,25.000000,25.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,59,59,4.50,0,1,0,1643841,8.743333,1.269167,1.855833,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,66.0,29.0
2,119,119,7.00,0,0,0,540296,510.660830,434.550840,0.000000,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,10.0
3,24,24,1.75,0,1,0,729600,319.575000,236.170840,0.000000,...,6.0,8.0,0.0,0.0,2.0,25.0,0.0,9.0,22.0,28.0
4,60,76,4.50,0,0,0,716836,71.818340,71.818340,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,45.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527899,4,4,0.25,0,0,0,1551651,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1527900,23,23,1.75,0,0,0,1584607,0.595833,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1527901,3,3,0.25,0,0,0,1246390,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1527902,24,24,1.75,0,1,0,695942,96.273330,63.232500,0.000000,...,12.0,2.0,0.0,0.0,6.0,0.0,1.0,0.0,NaN,NaN


In [31]:
import pandas as pd
from xgboost import XGBClassifier

# Assuming X_train and y_train are your training data, and X_test includes 'id' for patients
# X_test['id'] should have patient IDs, and X_test is your test data excluding the target variable

# Step 1: Train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Step 2: Predict the probabilities for the positive class (participation in preventive care)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Get probabilities for class 1 (participation)

# Step 3: Create a DataFrame with 'id' and 'score' (probability of participating in preventive care)
submission = pd.DataFrame({
    'id': X_test['id'],    # This is the patient ID column
    'score': y_pred_proba  # This is the predicted probability of participation (class 1)
})

# Step 4: Rank the patients based on the predicted score
submission['rank'] = submission['score'].rank(ascending=False, method='first').astype(int)

# Step 5: Sort by rank (optional)
submission = submission.sort_values(by='rank')

# Step 6: Save the submission file
submission.to_csv('submission.csv', index=False)

# Display the top of the submission DataFrame
print(submission.head())

/Users/jade/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:10:57] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              id     score  rank
1403537   399622  0.998955     1
1082446   645020  0.998916     2
886590   1537515  0.998860     3
1080831   719985  0.998694     4
1489309   101191  0.998665     5


In [32]:
import os
print(os.getcwd())

/Users/jade/Desktop/Humana/Training
